In [ ]:
# Importation des bibliothèques nécessaires pour le traitement des données et le calcul
import torch
from transformers import T5Tokenizer, T5EncoderModel
from utils import prottrans_T5_features  # Fonction utilitaire pour transformer les caractéristiques de séquences
from funcs import local_pairwise_alignment  # Fonction pour effectuer l'alignement local
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Affiche la version de PyTorch utilisée pour confirmer la version et faciliter le débogage
print(torch.__version__)

# Définit l'appareil sur lequel le modèle va s'exécuter (CPU dans ce cas), pour la compatibilité et la gestion des ressources
device = 'cpu'

# Nom du modèle pré-entraîné à utiliser, ici un modèle T5 pour les protéines, spécifique pour le traitement des séquences de protéines
prot_model_name = 'Rostlab/prot_t5_xl_uniref50'

# Charge le tokenizer associé au modèle T5, qui est utilisé pour convertir les séquences de protéines en un format compréhensible par le modèle
tokenizer = T5Tokenizer.from_pretrained(prot_model_name, legacy=True)

# Charge le modèle T5 et l'envoie sur le dispositif spécifié (CPU), prêt pour l'inférence
model = T5EncoderModel.from_pretrained(prot_model_name).to(device) 

# Définit la pénalité pour l'insertion de gaps dans l'alignement, qui influence la façon dont les séquences sont alignées en ajoutant des espaces
gap_penalty = 1

# Convertit les objets de séquences de protéines en chaînes de caractères pour traitement par le modèle
seq1 = str(sequences[0].seq)  # Convertit la séquence du premier SeqRecord en chaîne de caractères
seq2 = str(sequences[1].seq)  # Convertit la séquence du deuxième SeqRecord en chaîne de caractères

# Utilise le modèle T5 pour obtenir les caractéristiques des séquences, qui serviront de base pour l'alignement
features = prottrans_T5_features([seq1, seq2], tokenizer, model, device)

# Sélectionne les caractéristiques correspondant à chaque séquence pour l'alignement
f1 = features[0, :len(seq1)]  # Caractéristiques de la première séquence
f2 = features[1, :len(seq2)]  # Caractéristiques de la deuxième séquence

# Effectue l'alignement des deux séquences en utilisant les caractéristiques extraites et la pénalité de gap définie
aligned_seq1, aligned_seq2 = local_pairwise_alignment(seq1, seq2, f1, f2, gap_penalty)

# Simple print pour débogage ou confirmation d'exécution, indiquant que cette partie du code a été exécutée
print("coucou")

# Récupère les identifiants des séquences originales pour les utiliser dans l'enregistrement final
id1 = sequences[0].id  # ID de la première séquence
id2 = sequences[1].id  # ID de la deuxième séquence

# Nettoie les séquences alignées pour enlever les retours à la ligne et les retours chariot, qui peuvent causer des erreurs dans le fichier FASTA
aligned_seq1 = aligned_seq1.replace('\n', '').replace('\r', '')
aligned_seq2 = aligned_seq2.replace('\n', '').replace('\r', '')

# Crée des objets SeqRecord avec les séquences alignées et leurs identifiants, prêts pour l'enregistrement
r1 = SeqRecord(Seq(aligned_seq1), id=id1, description='')  # Crée un SeqRecord pour la première séquence
r2 = SeqRecord(Seq(aligned_seq2), id=id2, description='')  # Crée un SeqRecord pour la deuxième séquence

# Définit le chemin où le fichier FASTA sera enregistré
output_path = 'C:\\Users\\wrtmo\\Documents\\Internship\\pairwise_alignement\\data\\homstrad_data\\test_case1.fasta'

# Écrit les séquences alignées dans un fichier FASTA au chemin spécifié
SeqIO.write([r1, r2], output_path, 'fasta')

# Print additionnel pour confirmer la fin de l'exécution du script
print ("hello world")
